## 超參數優化（Fine Tune）
### 結構搜尋與建立模型

### 前置動作

接續FFNN的實作練習，先和當時做一樣的資料處理

In [8]:
#1 import 函式庫
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

In [9]:
#2 載入資料集
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

# 提取x, y
train_x = train.drop(['label'], axis=1)
train_y = train['label']

In [10]:
#3 分割資料集
kf = KFold(n_splits=4, shuffle=True, random_state=1202)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

In [11]:
#4 資料前處理
# 縮放像素值
tr_x, va_x = np.array(tr_x / 255.0), np.array(va_x / 255.0)

# One-hot編碼答案標籤
tr_y = to_categorical(tr_y, 10)
va_y = to_categorical(va_y, 10)

### 建立模型

編寫建立模型用的函式

In [12]:
#5 撰寫建立模型的函式
def Model(params):
    # 設定模型(第1層)
    model = Sequential()
    model.add(Input(shape=(tr_x.shape[1],)))
    model.add(Dense(int(params['first_units']), activation='relu'))
    model.add(Dropout(0.4))
    
    # 分層探索，決定是否要新增層
    if params['extra_layer'] == 'one':
        model.add(Dense(int(params['second_units']), activation='relu'))
    elif params['extra_layer'] == 'two':
        model.add(Dense(int(params['second_units']), activation='relu'))
        model.add(Dense(int(params['third_units']), activation='relu'))

    # 類別數量是固定的，不用探索Unit數量。
    model.add(Dense(10, activation='softmax'))

    # 編譯模型
    model.compile(loss='categorical_crossentropy',
                  optimizer=params['optimizer'],
                  metrics=['accuracy'])

    # 進行訓練
    result = model.fit(tr_x, tr_y,
                        epochs=10,
                        batch_size=100,
                        validation_data=(va_x, va_y),
                        verbose=0)

    # 輸出探索時的準確度
    val_acc = np.max(result.history['val_accuracy'])
    print('Accuracy in search:', val_acc)

    return {'loss': -val_acc, 'status': STATUS_OK, 'model': model}

### 進行超參數搜尋

定義搜尋空間並進行超參數搜尋

In [13]:
#6 定義搜尋空間
space = {'first_units': hp.choice('first_units', [500, 784]),
         'extra_layer': hp.choice('extra_layer', ['none', 'one', 'two']),
         'second_units': hp.choice('second_units', [100, 200]),
         'third_units': hp.choice('third_units', [25, 50]),
         'optimizer': hp.choice('optimizer', ['adam', 'rmsprop'])}

# 進行搜尋
trials = Trials()
best = fmin(fn=Model,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

print("Best hyperparameters:", best)

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

I0000 00:00:1748575377.565775      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1748575377.566445      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
I0000 00:00:1748575380.311714     105 service.cc:148] XLA service 0x7f78f8095790 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748575380.312679     105 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1748575380.312699     105 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1748575380.488433     105 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1748575381.656720     105 device_compiler.h:188] Compiled clust

Accuracy in search:                                   
0.9780952334403992                                    
Accuracy in search:                                                              
0.9768571257591248                                                               
Accuracy in search:                                                              
0.9790475964546204                                                               
Accuracy in search:                                                              
0.9774285554885864                                                               
Accuracy in search:                                                              
0.9752380847930908                                                               
Accuracy in search:                                                              
0.9795238375663757                                                               
Accuracy in search:                                                   

現在我們得到了最佳的超參數組合（模型結構）

### 驗證模型

用驗證集對模型進行驗證

In [14]:
#7 使用驗證資料來檢驗模型。
# 取得最佳模型
bestTrialIdx = np.argmin([trial['result']['loss'] for trial in trials.trials])
bestModel = trials.trials[bestTrialIdx]['result']['model']

# 輸出準確度最好的模型。
print(bestModel.summary())

# 使用驗證集評估模型
loss, accuracy = bestModel.evaluate(va_x, va_y)
print(f"val_loss: , {loss:.4f}")
print(f"val_acc: , {accuracy:.4f}")

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                     │ (None, 500)                 │         392,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 500)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 200)                 │         100,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 10)                  │           2,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 989,422 (3.77 MB)

 Trainable params: 494,710 (1.89 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 494,712 (1.89 MB)

None
329/329 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9786 - loss: 0.0853
val_loss: , 0.0915
val_acc: , 0.9763


到這邊模型的結構就確定了，接下來還可以調整細部的丟棄率、批次大小等

但由於`Hyperas`中`notebook_name='__notebook_source__`的關係，要新開一個Notebook 